# <center> 🚀SPACESHIP TITANIC -📊EDA + 27 DIFFERENT MODELS📈 </center>

<a id="3"></a>
# **<center><span style="color:#00BFC4;"> 1. 라이브러리 + 데이터 불러오기 </span></center>**

`IPython` : 표준 Python 셀보다 함수 객체가 더욱 향상

`lazypredict` : 머신러닝 자동화 오픈소스
- 여러 ML 모델을 불러올 수 있으나 파라미터 조정이 불가능함

In [ ]:
from IPython.display import clear_output #셀의 출력 결과를 지움
!pip3 install -U lazypredict
!pip3 install -U pandas #Upgrading pandas

clear_output()

`plotly` : python 데이터 시각화

> `plotly.express` : 데이터 값을 선으로 표현하는 그래프

> `plotly.graph_object` : 차트화

> `plotly.subplots` : 여러 그래프를 한 번에 설정하는 함수

--- 

`sklearn` : 결측치 

> `sklearn.impute` : 결측 데이터 

> `LabelEncoder` : 문자를 숫자

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px # 선
import matplotlib.pyplot as plt 
import plotly.graph_objects as go # 차트화
from plotly.subplots import make_subplots #여거 그래프 한번에 설정 -> subplot과 비교해보기



from sklearn.impute import SimpleImputer #sklearn.impute 패키지 : 결측 데이터 채워넣기 / SimpleImputer는 각 column의 결측치를 각 column의 평균값, 중간값, 최빈값 혹은 상수값으로 채우는 결측치 처리법입니다.
from sklearn.metrics import accuracy_score # accuracy_score( ) : 정답률 ( =정확도 )
from sklearn.preprocessing import LabelEncoder # 라벨인코더는 저저저저번부에 했죠??
from sklearn.model_selection import StratifiedKFold, train_test_split # 불균형한 분포도를 가진 레이블 데이터 집합을 위한 KFold 방식입니다
#교차검증이란? : 데이터 편중을 막기 위해서 별도의 세트로 구성된 학습 데이터 세트와 검증 데이터 세트에서 학습과 평가를 수행하는 것입니다.

from lightgbm import LGBMClassifier #알고리즘
import lazypredict  #머신러닝 모델 선택하기 ?!현재 분류와 회귀만 가능
                    #imputer = SimpleImputer(strategy="most_frequent")
                    # most_frequent : 최빈값, mean : 평균값, median : 중앙값
from lazypredict.Supervised import LazyClassifier

import time
import warnings
warnings.filterwarnings('ignore')


In [ ]:
train = pd.read_csv("../input/spaceship-titanic/train.csv")
test = pd.read_csv("../input/spaceship-titanic/test.csv")
submission = pd.read_csv("../input/spaceship-titanic/sample_submission.csv")

RANDOM_STATE = 12 
FOLDS = 5
STRATEGY = 'median'

<a id="3"></a>
# **<center><span style="color:#00BFC4;"> 2. 데이터 파악 </span></center>**


- `PassengerId` - 각 승객의 고유한 ID
                  - gggg : 같이 가는 그룹
                  - pp : 그룹 안의 번호

- `HomePlanet` - 출발지

- `CryoSleep` - 여행 중 승객이 동면상태에 드는 여부

- `Cabin` - 머무는 객실

- `Destination` - 목적지

- `Age` - 나이

- `VIP` - VIP

- `RoomService`, FoodCourt, ShoppingMall, Spa, VRDeck - 승객이 우주선에서 지불해야할 금액

- `Name` - 처음, 끝 이름

- `Transported` - 다른 차원으로 이동되었는지


### (1) Train
`행` - 8693개
`열` - 14개
`데이터` - 119378개
`결측값` - 2324개

In [ ]:
train.head()

In [ ]:
print(f'\033[94mNumber of rows in train data: {train.shape[0]}')
print(f'\033[94mNumber of columns in train data: {train.shape[1]}')
print(f'\033[94mNumber of values in train data: {train.count().sum()}')
print(f'\033[94mNumber missing values in train data: {sum(train.isna().sum())}')

### (2) 열의 결측값

In [ ]:
print(f'\033[94m')
print(train.isna().sum().sort_values(ascending = False))

In [ ]:
train.describe()

### (2) Test
`행` - 4277개
`열` - 13개
`데이터` - 54484개
`결측값` - 1117개

In [ ]:
test.head()

In [ ]:
print(f'\033[94mNumber of rows in test data: {test.shape[0]}')
print(f'\033[94mNumber of columns in test data: {test.shape[1]}')
print(f'\033[94mNumber of values in train data: {test.count().sum()}')
print(f'\033[94mNo of rows with missing values  in test data: {sum(test.isna().sum())}')

`행` - 4277개
`열` - 13개
`데이터` - 54484개
`결측값` - 1117개

In [ ]:
print(f'\033[94m')
print((test.isna().sum().sort_values(ascending = False)))

In [ ]:
test.describe()

### (3) Submission
`행` → PassengerId - Transported

In [ ]:
submission.head()

<a id="4"></a>
# **<center><span style="color:#00BFC4;"> EDA </span></center>**

<a id="4.1"></a>
## <span style="color:#e76f51;"> 데이터 분석 </span>

In [ ]:
train.drop(["PassengerId"] , axis = 1 , inplace = True)
test.drop(["PassengerId"] , axis = 1 , inplace = True)
TARGET = 'Transported'
FEATURES = [col for col in train.columns if col != TARGET]
RANDOM_STATE = 12 

In [ ]:
train.iloc[:, :-1].describe().T.sort_values(by='std' , ascending = False)\
                     .style.background_gradient(cmap='GnBu')\
                     .bar(subset=["max"], color='#BB0000')\
                     .bar(subset=["mean",], color='green')

<a id="4.7"></a>
## <span style="color:#e76f51;"> (1) 분포 - 결측값   </span>

<div class="alert alert-block alert-info" style="font-size:14px; font-family:verdana; line-height: 1.7em;">
    📌 &nbsp;<b><u>Observations in Null Value Distribution :</u></b><br>
 
* <i> The maximum of missing value in an row is <b><u>3</u></b> and the lowest is <b><u>no missing value</u></b></i><br>
* <i> Interestingly, the missing value distribution (row basis) is quite the same between <b><u>train</u></b> and <b><u>test</u></b> dataset. </i><br>
* <i> There are around <b><u>76%</u></b> of the observations (row basis) that has no missing values.  </i><br>
* <i> Rest <b><u>24%</u></b> of the observations (row basis) that have <b><u>1 to 3</u></b>  missing values</i><br>
</div>

<a id="4.2.1"></a>
### <span style="color:#e76f51;">(1) 행 결측값  </span>

In [ ]:
# TEST 
test_null = pd.DataFrame(test.isna().sum()) #결측값 확인
test_null = test_null.sort_values(by = 0 ,ascending = False) # 내림차순으로 정렬

# TRAIN 
train_null = pd.DataFrame(train.isna().sum()) #결측값 확인
train_null = train_null.sort_values(by = 0 ,ascending = False)[:-1] # 내림차순으로 정렬


# 그래프 생성
fig = make_subplots(rows=1, 
                    cols=2,
                    column_titles = ["Train Data", "Test Data"] ,   # train set과 test 셋의 컬럼을 가져온다
                    x_title="Missing Values")

fig.add_trace(go.Bar(x=train_null[0],      #데이터(df)를 지정해 넣고 x, y 옵션에 해당 데이터의 컬럼 값을 주면 된다.
                     y=train_null.index,
                     orientation="h",
                    marker=dict(color=[n for n in range(12)], 
                                line_color='rgb(0,0,0)' , 
                                line_width = 2,
                                coloraxis="coloraxis")),  
              1, 1)
#color 값은 지정한 컬럼에 따라 컬러를 다르게 주겠다는 의미
#특정 값을 텍스트로 따로 표기하고 싶다면 text 옵션에 표기하고자 하는 컬럼값을 넣어주면 된다.

fig.add_trace(go.Bar(x=test_null[0], 
                     y=test_null.index,
                     orientation="h",
                    marker=dict(color=[n for n in range(12)], 
                                line_color='rgb(0,0,0)', 
                                line_width = 2,
                                coloraxis="coloraxis")),
              1, 2)

fig.update_layout(showlegend=False, title_text="Column wise Null Value Distribution", title_x=0.5)
#fig.update_layout() 은 그래프 생성 완료 후 layout 정보를 업데이트 해주는 함수

# bar plot
## 한개이상의 차트를 하나의 피규어에 작성할때 go.bar 를 주로 사용한다

## 이 외에도 barchart group vertical + line plot barchart (horizontal) 등 다양항 bar chart가 있다.

<a id="4.7.2"></a>
### <span style="color:#e76f51;"> (2) 열 결측값 </span>

In [ ]:
# TRAIN 결측값 합
missing_train_row = train.isna().sum(axis=1)
missing_train_row = pd.DataFrame(missing_train_row.value_counts()/train.shape[0]).reset_index()

# TEST 결측값 합
missing_test_row = test.isna().sum(axis=1)
missing_test_row = pd.DataFrame(missing_test_row.value_counts()/test.shape[0]).reset_index()


missing_train_row.columns = ['no', 'count']
missing_test_row.columns = ['no', 'count']
missing_train_row["count"] = missing_train_row["count"]*100
missing_test_row["count"] = missing_test_row["count"]*100


fig = make_subplots(rows=1, 
                    cols=2,
                    column_titles = ["Train Data", "Test Data"] ,
                    x_title="Missing Values",)

fig.add_trace(go.Bar(x=missing_train_row["no"], 
                     y=missing_train_row["count"]  ,
                    marker=dict(color=[n for n in range(4)], 
                                line_color='rgb(0,0,0)' ,
                                line_width = 3
                                ,coloraxis="coloraxis")),
              1, 1)
fig.add_trace(go.Bar(x= missing_test_row["no"], 
                     y=missing_test_row["count"],
                    marker=dict(color=[n for n in range(4)], 
                                line_color='rgb(0,0,0)',
                                line_width = 3 ,
                                coloraxis="coloraxis")),
              1, 2)
fig.update_layout(showlegend=False, title_text="Row wise Null Value Distribution", title_x=0.5)

<a id="4.2"></a>
## <span style="color:#e76f51;"> 분포 - Continuos, Categorical Data </span>

In [ ]:
# train, test featrue 결합
df = pd.concat([train[FEATURES], test[FEATURES]], axis=0)

text_features = ["Cabin", "Name"]
cat_features = [col for col in FEATURES if df[col].nunique() < 25 and col not in text_features ]
cont_features = [col for col in FEATURES if df[col].nunique() >= 25 and col not in text_features ]

del df
print(f'\033[94mTotal number of features: {len(FEATURES)}')
print(f'\033[94mNumber of categorical features: {len(cat_features)}')
print(f'\033[94mNumber of continuos features: {len(cont_features)}')
print(f'\033[94mNumber of text features: {len(text_features)}')


# 라벨링 후 그래프 생성
# (1) Categorical - 텍스트 X인 25미만의 고유값
# (2) Continuous - 텍트스 X인 25이상의 고유값
# (3) Text - 텍스트 형식의 데이터, Cabin, Name
labels=['Categorical', 'Continuos', "Text"]
values= [len(cat_features), len(cont_features), len(text_features)]
colors = ['#DE3163', '#58D68D']

# 그래프로 생성
fig = go.Figure(data=[go.Pie(
    labels=labels, 
    values=values, pull=[0.1, 0, 0 ],
    marker=dict(colors=colors, 
                line=dict(color='#000000', 
                          width=2))
)])
fig.show()

<a id="4.3"></a>
## <span style="color:#e76f51;"> 1. 분포 - Continous Features </span>

### <span style="color:#e76f51;">  나이 분포 </span>



In [ ]:
# train, test의 나이 
train_age = train.copy()
test_age = test.copy()
train_age["type"] = "Train"
test_age["type"] = "Test"

# 둘의 값 합 - 히스토그램으로 출력
ageDf = pd.concat([train_age, test_age])
fig = px.histogram(data_frame = ageDf, 
                   x="Age",
                   color= "type",
                   color_discrete_sequence =  ['#58D68D','#DE3163'],
                   marginal="box",
                   nbins= 100,
                    template="plotly_white"
                )
fig.update_layout(title = "Distribution of Age" , title_x = 0.5)
fig.show()

<a id="4.4"></a>
## <span style="color:#e76f51;">  2. 분포 - Categorical Features </span>

In [ ]:
if len(cat_features) == 0 :
    print("No Categorical features")
else:
    ncols = 2
    nrows = 2

    fig, axes = plt.subplots(nrows, ncols, figsize=(18, 10))
    for r in range(nrows):
        for c in range(ncols):
            col = cat_features[r*ncols+c]
            sns.countplot(train[col],ax = axes[r,c] ,palette = "viridis", label='Train data')
            sns.countplot(test[col],ax = axes[r,c] ,palette = "magma", label='Test data')
            axes[r,c].legend()
            axes[r,c].set_ylabel('')
            axes[r,c].set_xlabel(col, fontsize=20)
            axes[r,c].tick_params(labelsize=10, width=0.5)
            axes[r,c].xaxis.offsetText.set_fontsize(4)
            axes[r,c].yaxis.offsetText.set_fontsize(4)
    plt.show()

<a id="4.5"></a>
## <span style="color:#e76f51;"> 3. 분포 - Target (도착) </span>

In [ ]:
target_df = pd.DataFrame(train[TARGET].value_counts()).reset_index()
target_df.columns = [TARGET, 'count']
fig = px.bar(data_frame =target_df, 
             x = TARGET,
             y = 'count'
            ) 
fig.update_traces(marker_color =['#58D68D','#DE3163'], 
                  marker_line_color='rgb(0,0,0)',
                  marker_line_width=2,)
fig.update_layout(title = "Target Distribution",
                  template = "plotly_white",
                  title_x = 0.5)
print("\033[94mPercentage of Transported = 0: {:.2f} %".format(target_df["count"][0] *100 / train.shape[0]))
print("\033[94mPercentage of Transported = 1: {:.2f} %".format(target_df["count"][1]* 100 / train.shape[0]))
fig.show()

<a id="4.6"></a>
## <span style="color:#e76f51;"> 상관관계 </span>

In [ ]:
fig = px.imshow(train.corr() ,text_auto=True, aspect="auto" , color_continuous_scale = "viridis")
fig.show()

<a id="4.2"></a>
## <span style="color:#e76f51;"> 데이터 전처리 </span>

In [ ]:
# 전처리 행 - 나이, 이용서비스(푸드코트, 쇼핑몰, 스파, VR, 룸서비스)
imputer_cols = ["Age", "FoodCourt", "ShoppingMall", "Spa", "VRDeck", "RoomService"]

# 사이킷런 - SimpleImputer .. Nan 값을 채우기 위함
imputer = SimpleImputer(strategy = STRATEGY)
imputer.fit(train[imputer_cols])

# DataFrame 생성 
train[imputer_cols] = imputer.transform(train[imputer_cols])
test[imputer_cols] = imputer.transform(test[imputer_cols])

# fillna - 결측값 변경
train["HomePlanet"].fillna('Z', inplace=True)
test["HomePlanet"].fillna('Z', inplace=True)

In [ ]:
label_cols = ["HomePlanet", "CryoSleep", "Cabin", "Destination", "VIP"]
def label_encoder(train, test, columns):
    for col in columns:
        train[col] = train[col].astype(str)
        test[col] = test[col].astype(str)
        train[col] = LabelEncoder().fit_transform(train[col])
        test[col] = LabelEncoder().fit_transform(test[col])
    return train, test

train, tset = label_encoder(train, test, label_cols)

<a id="6"></a>
# **<center><span style="color:#00BFC4;">모델링 </span></center>**

`clf` - classifiers 의 약자

`verbose` - 함수 수행시 발생하는 상세정보 출력여부 // 0 출력 안함

`ignore_warnings` - 경고 출력 // Ture

`custom_metric` - 기능이 제공, 사용자 정의 평가 매트릭을 기반으로 모델이 평가됨

`predictions` - True 면, 모델의 예측이 데이터 프레임으로 반환


In [ ]:
clf = LazyClassifier(verbose=0,
                     ignore_warnings=True,
                     custom_metric=None,
                     predictions=False,
                     random_state=12,
                     classifiers='all')

models, predictions = clf.fit(X_train , X_test , y_train , y_test)
clear_output()

### 모델 별 성능

`Accuracy` : 정확도

`Balanced Accuracy` : 불균형 데이터 셋에 과장된 성능 추정 방지

`ROC AUC` : 다양한 임계값 모델의 분류 성능에 대한 측정

`F1 Score` : 점수

`Time Taken` : 걸리는 시간

In [183]:
models[:15]

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,


### <span style="color:#e76f51;"> 시각화 </span>

### 1. 모델 별 정확도

In [ ]:
line = px.line(data_frame= models ,y =["Accuracy"] , markers = True)
line.update_xaxes(title="Model",
              rangeslider_visible = False)
line.update_yaxes(title = "Accuracy")
line.update_traces(line_color="red")
line.update_layout(showlegend = True,
    title = {
        'text': 'Accuracy vs Model',
        'y':0.94,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

line.show()

### 2. 모델 별 ROC AUC / F1Score

In [ ]:
line = px.line(data_frame= models ,y =["ROC AUC" , "F1 Score"] , markers = True)
line.update_xaxes(title="Model",
              rangeslider_visible = False)
line.update_yaxes(title = "ROC AUC SCORE")
line.update_layout(showlegend = True,
    title = {
        'text': 'ROC AUC and F1 Score vs Model',
        'y':0.94,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

line.show()

### 3. 모델 별 측정된 시간

In [ ]:
line = px.line(data_frame= models ,y =["Time Taken"] , markers = True)
line.update_xaxes(title="Model",
              rangeslider_visible = False)
line.update_yaxes(title = "Time(s)")
line.update_traces(line_color="purple")
line.update_layout(showlegend = True,
    title = {
        'text': 'TIME TAKEN vs Model',
        'y':0.94,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

line.show()

<a id="6.2"></a>
## <span style="color:#e76f51;"> LGBM Classifier(5 FOLDS)  : </span>

In [ ]:
lgb_params = {
    'objective' : 'binary',
    'n_estimators' :50,  #결정 트리의 개수
    'learning_rate' : 0.08  #학습률
}
# Tree 기반 학습 알고리즘

lgb_predictions = 0
lgb_scores = []
lgb_fimp = []
LGBM_FEATURES = list(train.columns)[:-1]

# StratifiedKFold : 불균형한 분포도를 가진 레이블 데이터 집합을 위한 KFold 방식
skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=RANDOM_STATE) 
for fold, (train_idx, valid_idx) in enumerate(skf.split(train[LGBM_FEATURES], train[TARGET])):
    print(f'\033[94m')
    print(10*"=", f"Fold={fold+1}", 10*"=")
    start_time = time.time()
    
    X_train, X_valid = train.iloc[train_idx][LGBM_FEATURES], train.iloc[valid_idx][LGBM_FEATURES]
    y_train , y_valid = train[TARGET].iloc[train_idx] , train[TARGET].iloc[valid_idx]
    
    model = LGBMClassifier(**lgb_params)
    model.fit(X_train, y_train,verbose=0)
    
    preds_valid = model.predict(X_valid)
    acc = accuracy_score(y_valid,  preds_valid)
    lgb_scores.append(acc)
    run_time = time.time() - start_time
    
    fim = pd.DataFrame(index=LGBM_FEATURES,
                 data=model.feature_importances_,
                 columns=[f'{fold}_importance'])
    lgb_fimp.append(fim)
    
    print(f"Fold={fold+1}, Accuracy score: {acc:.2f}%, Run Time: {run_time:.2f}s")
    test_preds = model.predict(test[LGBM_FEATURES]) 
    lgb_predictions += test_preds/FOLDS
print("")
print("Mean Accuracy :", np.mean(lgb_scores))

### <span style="color:#e76f51;">Feature Importance </span>

In [ ]:
lgbm_fis_df = pd.concat(lgb_fimp, axis=1).head(15)
lgbm_fis_df.sort_values('1_importance').plot(kind='barh', figsize=(15, 10), # sort_values : 값 기준 정렬, 
                                             # 1_importance : 값 중요도..?
                                       title='Feature Importance Across Folds')
plt.show()

<a id="7"></a>
# **<center><span style="color:#00BFC4;">Submission </span></center>**

In [ ]:
submission[TARGET] = lgb_predictions.astype("bool") # astype : 데이터 타입 변경
submission.to_csv("submission.csv",index=False) # 결과 저장
submission.head()